# PLS regression analysis tutorial

### Import the required packages

In [15]:
import pandas as pds
import numpy as np
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_predict, permutation_test_score
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
import plotly.express as px

### Load the example dataset using pandas

In [2]:
lcMSData = pds.read_csv('./Data/Dementia_RPOS_XCMS.csv')

In [3]:
# Read the retention time and m/z value from feature names
featuresData = pds.DataFrame([(float(x.split('_')[0]), float(x.split('_')[1][:-3])) for x in lcMSData.columns[11:]], columns=['Rt', 'mz'])
featuresData['Rt'] = featuresData['Rt']/60
medianSpectrum = np.median(lcMSData.iloc[:, 11:].values, axis=0)

# Use log of median spectra as intensity value for the scatterplot
featuresData['Median'] = np.log(medianSpectrum + 1)
#featuresData['Median'] = medianSpectrum 

## PLS regression model

Fit a PLS regression model to predict age (Y) using the LC-MS profiles.

In [4]:
XDataMatrix = lcMSData.iloc[:, 11:].values

logXDataMatrix = np.log(XDataMatrix + 1)

YAge = lcMSData['Age'].values

In [5]:
# Scaling will be handled with sckit-learn StandardScaler()
plsModel = Pipeline(steps=[('uv_scale', StandardScaler()), ('PLS', PLSRegression(n_components=2, scale=False))])

# To switch off scaling use this line instead
# plsModel = PLSRegression(n_components=2, scale=False)

# Fit the PLS model
plsModel.fit(logXDataMatrix, YAge)

Pipeline(steps=[('uv_scale', StandardScaler()),
                ('PLS', PLSRegression(scale=False))])

In [6]:
plsModel

Pipeline(steps=[('uv_scale', StandardScaler()),
                ('PLS', PLSRegression(scale=False))])

### PLS regression prediction

The PLS regression model predicts age (y) based on the metabolic variables in X. The r-squared measure $R^2_Y$ quantifies the model's goodness of fit. 

In [39]:
predictFrame = pds.DataFrame(np.c_[plsModel.predict(logXDataMatrix), YAge], columns=['Predicted', 'Age'])

fig = px.scatter(predictFrame, x="Predicted", y="Age", render_mode='webgl', 
                labels={"Predicted": "PLS predicted Age",
                        "Age": "Age"}, 
                template='plotly_white')

fig.show()

"Variance of Y explained (R2Y): {0}".format(plsModel.score(logXDataMatrix, YAge))

'Variance of Y explained (R2Y): 0.8748721101266395'

### PLS scores plot
Lets's now examine the PLS scores plot, the lower dimensional projections.

In [40]:
T_scores = plsModel.transform(XDataMatrix)

# Assemble a pandas data frame with the scores for each component and then combine with study variables
plsResultsDFrame = pds.DataFrame(T_scores, columns=['PLS T' + str(x+1) for x in range(T_scores.shape[1])])
plsResultsDFrame = pds.concat([lcMSData.loc[:, ['Subject ID', 'Sample ID', 'Age', 'Gender', 'Run Order', 'Acquisition batch']], plsResultsDFrame], axis=1)

In [41]:
fig = px.scatter(plsResultsDFrame, x="PLS T1", y="PLS T2", color="Age",
                 render_mode='webgl', color_continuous_scale='haline',
                template='plotly_white')

fig.show()

### Model interpretation and variable importance

The main parameters to assess variable importance in a PLS model are the weights $w$, the loadings $p$, and regression coefficients $\beta$.

The weights convey the covariance shared directly by the X variables with Y. Each component has its own weight vector, with values varying between -1 (strong negative-covariance) and 1 (strong covariance), with 0 meaning no association/covariance. The weight vector of the first component (which explains the most variation in Y) is the primary weight vector to analyze when interpreting the main variables of X associed with Y.

Loadings should be used like PCA loadings, to interpret the directions in the PLS score plots. Each PLS component has its own loading vector.

The regression coefficients ($\beta$) have a similar interpretation as regression coefficients in a multivariate/multiple linear regression. There is only one vector of coefficients for the entire model.

In [42]:
# Loadings p - x_loadings_
# Weights w - x_weights_
# Regression coefficients - coef_

fig = px.scatter(featuresData, x="Rt", y="mz", color=plsModel['PLS'].x_weights_[:, 0], render_mode='webgl', 
                color_continuous_scale='RdBu', color_continuous_midpoint=0,
                labels={"Rt": "Retention time (min)",
                        "mz": "m/z"}, 
                template='plotly_white')

fig.show()

### Model validation and overfitting

We will use cross-validation (7-Fold cross-validation) to select the optimal number of PLS components to use and obtain more reliable model performance estimates. The r-squared values will be calculated on external test set data (data not used to fit the model).

In [43]:
# Set up the PLS model with 'uv' scaling
plsModel = Pipeline(steps=[('uv_scale', StandardScaler()), ('PLS', PLSRegression(scale=False))])

In [44]:
# Define a function to fit and cross-validate a PLS model
def crossValidate_PLS(x, y, n_components=2, scale=True, cv=KFold(7, shuffle=True)):
    
    if scale is True:
        plsModel = Pipeline(steps=[('uv_scale', StandardScaler()), 
                                   ('PLS', PLSRegression(n_components=n_components, scale=False))])
    else:
        plsModel = Pipeline(steps=[('PLS', PLSRegression(n_components=n_components, scale=False))])
        
    cvResults = {'r2':[]}
    
    # Iterate through CV rounds
    for trainIdx, testIdx in cv.split(x):
        
        # Fit the PLS model on training set
        plsModel.fit(x[trainIdx, :], y[trainIdx])
        # test
        cvResults['r2'].append(r2_score(y[testIdx], plsModel.predict(x[testIdx, :])))
        
    cvResults = {key: np.array(value) for key, value in cvResults.items()}
    return pds.DataFrame(cvResults)

In [45]:
cvResults = crossValidate_PLS(logXDataMatrix, YAge, n_components=2, scale=True, cv=KFold(7, shuffle=True))

The result of the 7-Fold CV process is 7 instances of the classifier performance metrics chosen (r2). 

In [46]:
cvResults

,r2
0,0.258750
1,0.392864
2,0.294746
3,0.320178
4,0.044946
5,0.291834
6,0.345840


### Selecting the optimal number of components with cross-validation

To select the number of PLS components to use, the CV procedure should be applied to models with a varying number of components. The CV metrics will be then used to generate a "scree plot".

In [47]:
maxNComponents = 10

screePLS = [crossValidate_PLS(logXDataMatrix, YAge, n_components=x, scale=True, cv=KFold(7, shuffle=True)) for x in range(1, maxNComponents + 1)]

In [48]:
cvPLS_DFrame = list()

for ncomp, cv in enumerate(screePLS):
    currentNComp = pds.DataFrame(cv)
    currentNComp['Ncomp'] = ncomp + 1
    cvPLS_DFrame.append(currentNComp)
    
cvPLS_DFrame = pds.concat(cvPLS_DFrame, axis=0)


In [49]:
fig = px.box(cvPLS_DFrame, x='Ncomp', y='r2', # points="all",
             labels={"Ncomp": "Number of components",
                        "auc": "ROC AUC"}, template='plotly_white')

fig.show()

The gains in model performance after 3 components are small, and the r2 values become slightly more unstable. We will select 3 as the optimal number of components.

### Fit model with optimal number of PLS components

In [26]:
plsModel = Pipeline(steps=[('uv_scale', StandardScaler()), ('PLS', PLSRegression(n_components=3))])

# Fit the PLS Model
plsModel.fit(logXDataMatrix, YAge)

Pipeline(steps=[('uv_scale', StandardScaler()),
                ('PLS', PLSRegression(n_components=3))])

In [27]:
cvResults = crossValidate_PLS(logXDataMatrix, YAge, n_components=3, scale=True, cv=KFold(7, shuffle=True))

These cross-validated metrics are better estimates of the expected model performance.

In [28]:
pds.DataFrame(np.c_[cvResults.mean(), cvResults.std()], columns=['Mean', 'Stdev'], index=cvResults.columns)

,Mean,Stdev
r2,0.29276,0.147749


In [32]:
"Variance of Y explained (R2Y), estimated without CV: {0}".format(plsModel.score(logXDataMatrix, YAge))

'Variance of Y explained (R2Y), estimated without CV: 0.8748721101266395'

The CV estimated r2 is much smaller - 0.29 vs 0.87...

Lets now revisit the model predictions plot using cross-validated predictions.

In [33]:
cvPredictions = cross_val_predict(plsModel, logXDataMatrix, YAge, cv=KFold(n_splits=7, shuffle=True))

In [34]:
predictFrame = pds.DataFrame(np.c_[cvPredictions, YAge], columns=['Predicted', 'Age'])

fig = px.scatter(predictFrame, x="Predicted", y="Age", render_mode='webgl', 
                labels={"Predicted": "PLS predicted Age",
                        "Age": "Age"}, 
                template='plotly_white')

fig.show()

**Summary**: Although with cross-validation we have evidence that there is some robust predictive association between metabolic profile and age, the strength of this association (measured by the $R^2_y$) is overestimated.

### Permutation randomisation test

A final and very important method for model validation is the permutation randomization test. In a permutation randomisation test, the model will be refitted and assessed multiple times, but each time with the Y randomly permuted to destroy any relationship between X & Y. This allows us to assess what sort of model we can get when there really is no relationship between the two data matrices, and calculate the likelihood of obtaining a model with predictive performance as good as the non-permuted model by chance alone.

During this test, the number of components, scaling, type of cross-validation employed, and any other modeling choice is kept constant. In each randomization, the model is refitted, and the performance and model validation metric recorded. This enables the generation of permuted null distributions for these metrics, which can be used to obtain an empirical p-value for their significance.

**Note**: Running the permutation test with a large number of permutation randomizations (for example, 1000) is expected to take a considerable ammount of time (> 30 mins on a laptop).

In [50]:
permutationTest = permutation_test_score(plsModel, logXDataMatrix, YAge, cv=KFold(n_splits=7, shuffle=True), n_permutations=250, n_jobs=4)

Histogram of results from permuted (null) models. The vertical line represents the cross-validated $R^2_y$ value obtained in the "real" model. **Note**: The numerical precision of the p-value estimate is dependent on the number of permutations used.

In [51]:
fig = px.histogram(pds.DataFrame(permutationTest[1], columns=['r2']), x='r2', nbins=20)

fig.add_vline(x=permutationTest[0], line_dash="dash")
fig.show()
"Permutation p-value ~ {0}".format(permutationTest[2])

'Permutation p-value ~ 0.00398406374501992'